# 📚 Análise de Rede de E-mails - Google Colab (Execução Automática)
Lista de Exercícios 2 - Ciência de Dados
Universidade Federal de Alagoas (UFAL)
Aluno: Kleber José Araujo Galvão Filho

## 🔼 Upload do Dataset
carregue a base de dados ".txt.gz"

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving email-Eu-core-temporal.txt.gz to email-Eu-core-temporal.txt.gz


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import gzip
import os
import numpy as np
import random
from statsmodels.tsa.seasonal import seasonal_decompose
from networkx.algorithms.community import louvain_partitions
from scipy import stats

In [ ]:
import pandas as pd
import gzip
import os

def carregar_dados(caminho_arquivo_gz, caminho_saida_csv="email-Eu-core-temporal.csv"):
    """
    Descomprime o arquivo .gz, lê o arquivo de texto e converte para CSV.
    Args:
        caminho_arquivo_gz (str): Caminho para o arquivo .gz.
        caminho_saida_csv (str): Caminho onde o CSV será salvo.
    Returns:
        pd.DataFrame: DataFrame com colunas 'origem', 'destino', 'timestamp'.
    """
    if os.path.exists(caminho_saida_csv):
        print(f"Carregando CSV existente: {caminho_saida_csv}")
        dados = pd.read_csv(caminho_saida_csv)
        return dados

    try:
        with gzip.open(caminho_arquivo_gz, 'rt') as arquivo:
            dados = pd.read_csv(arquivo, sep='\s+', names=['origem', 'destino', 'timestamp'], engine='python')
        dados.to_csv(caminho_saida_csv, index=False)
        print(f"Dados convertidos e salvos como: {caminho_saida_csv}")
        print(f"Dataset carregado com {len(dados)} arestas.")
        return dados
    except FileNotFoundError:
        print(f"Erro: Arquivo {caminho_arquivo_gz} não encontrado.")
        raise
    except Exception as e:
        print(f"Erro ao processar o arquivo: {str(e)}")
        raise

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import os

def exercicio_2(dados, tipo_layout="spring"):
    """
    Visualiza a rede social direcionada com diferentes layouts.
    Args:
        dados (pd.DataFrame): DataFrame com as arestas da rede.
        tipo_layout (str): Tipo de layout ('spring', 'kamada', 'circular').
    """
    grafo = nx.DiGraph()
    arestas = dados[['origem', 'destino']].values
    grafo.add_edges_from(arestas)

    plt.figure(figsize=(10, 10))

    if tipo_layout == "spring":
        posicao = nx.spring_layout(grafo, k=0.15, iterations=20)
        nome_arquivo = os.path.join("figuras", "exercicio_02_rede_spring.png")
        titulo = "Rede de E-mails Direcionada (Spring Layout)"
    elif tipo_layout == "kamada":
        posicao = nx.kamada_kawai_layout(grafo)
        nome_arquivo = os.path.join("figuras", "exercicio_02_rede_kamada.png")
        titulo = "Rede de E-mails Direcionada (Kamada-Kawai Layout)"
    elif tipo_layout == "circular":
        posicao = nx.circular_layout(grafo)
        nome_arquivo = os.path.join("figuras", "exercicio_02_rede_circular.png")
        titulo = "Rede de E-mails Direcionada (Circular Layout)"
    else:
        print("Layout inválido. Usando spring como padrão.")
        posicao = nx.spring_layout(grafo, k=0.15, iterations=20)
        nome_arquivo = os.path.join("figuras", "exercicio_02_rede_spring.png")
        titulo = "Rede de E-mails Direcionada (Spring Layout)"

    nx.draw(grafo, posicao, node_size=50, arrows=True, with_labels=False)
    plt.title(titulo)
    plt.savefig(nome_arquivo)
    plt.close()
    print(f"Visualização da rede salva como '{nome_arquivo}'.")

In [ ]:
import pandas as pd
import networkx as nx

def exercicio_3(dados):
    """
    Calcula a média dos menores caminhos na rede direcionada e analisa conectividade e eficiência.
    Args:
        dados (pd.DataFrame): DataFrame com as arestas da rede.
    Returns:
        float: Média dos menores caminhos (maior componente conexo ou grafo completo).
    """
    # Usa grafo direcionado para refletir a rede de e-mails
    grafo = nx.DiGraph()
    arestas = dados[['origem', 'destino']].values
    grafo.add_edges_from(arestas)

    # Verifica se o grafo é fortemente conectado (para DiGraph)
    if nx.is_strongly_connected(grafo):
        media_caminhos = nx.average_shortest_path_length(grafo)
        print(f"Média dos menores caminhos (grafo completo): {media_caminhos:.4f}")
    else:
        # Usa o maior componente fortemente conexo
        maior_componente = max(nx.strongly_connected_components(grafo), key=len)
        grafo_componente = grafo.subgraph(maior_componente).copy()
        media_caminhos = nx.average_shortest_path_length(grafo_componente)
        print(f"Média dos menores caminhos (maior componente fortemente conexo): {media_caminhos:.4f}")

    # Interpretação detalhada
    print("\nAnálise da conectividade e eficiência:")
    if media_caminhos < 6:
        print(f"A média de {media_caminhos:.4f} indica alta conectividade (característica de mundo pequeno).")
        print("E-mails tendem a alcançar destinatários com poucos intermediários, sugerindo comunicação eficiente.")
    else:
        print(f"A média de {media_caminhos:.4f} sugere conectividade moderada a baixa.")
        print("A comunicação pode ser menos eficiente, exigindo mais intermediários para e-mails entre nós distantes.")
    print("Em redes sociais, médias abaixo de 6 são comuns, como no conceito de 'seis graus de separação'.")

    return media_caminhos

In [ ]:
import pandas as pd
import networkx as nx

def exercicio_4(dados):
    """
    Calcula os top 5 nós com base em diferentes métricas de centralidade.
    Args:
        dados (pd.DataFrame): DataFrame com as arestas da rede.
        metrica (str): 'betweenness', 'degree', ou 'closeness'.
    Returns:
        list: Lista de tuplas (nó, valor) para os 5 principais nós.
    """
    grafo = nx.DiGraph()
    arestas = dados[['origem', 'destino']].values
    grafo.add_edges_from(arestas)

    centralidade = nx.betweenness_centrality(grafo)
    nome_metrica = "centralidade de intermediação"

    top_5 = sorted(centralidade.items(), key=lambda x: x[1], reverse=True)[:5]
    print(f"Top 5 nós por {nome_metrica}:")
    for no, valor in top_5:
        print(f"Nó {no}: {valor:.4f}")
        print("Nós com alta intermediação conectam grupos distintos.")

    print("\nInterpretação no contexto da instituição de pesquisa:")
    print("A centralidade de intermediação mede a frequência com que um nó aparece nos menores caminhos entre outros nós na rede de e-mails.")
    print("Nós com alta centralidade são cruciais para o fluxo de informações, funcionando como pontes entre diferentes grupos ou departamentos.")
    print("\nPossíveis papéis desses nós incluem:")
    print("- **Líderes de pesquisa**: Conectam equipes interdisciplinares, promovendo colaboração em projetos acadêmicos.")
    print("- **Administradores**: Coordenam comunicações institucionais, como anúncios ou decisões estratégicas.")
    print("- **Sistemas centrais**: Servidores de e-mail ou newsletters que disseminam informações amplamente.")
    print("\nEstrutura da rede de comunicação:")
    print("A presença de nós com alta centralidade sugere uma rede integrada, onde a colaboração entre departamentos é facilitada.")
    print("No entanto, a rede é centralizada, dependendo de poucos nós críticos. Isso implica:")
    print("- **Eficiência**: Informações fluem rapidamente através desses nós, agilizando a troca de conhecimento.")
    print("- **Vulnerabilidade**: A sobrecarga ou ausência desses nós pode fragmentar a comunicação, isolando grupos e dificultando a colaboração.")
    print("\nImplicações:")
    print("Esses nós são pontos estratégicos para a instituição, mas também pontos de risco. Estratégias como descentralizar comunicações ou criar redundâncias podem mitigar dependências.")

    return top_5

In [ ]:
import pandas as pd
import networkx as nx
from networkx.algorithms.community import louvain_partitions

def exercicio_5(dados):
    """
    Detecta comunidades usando o algoritmo de Louvain do NetworkX e encontra o nó principal por maior grau de entrada.
    Args:
        dados (pd.DataFrame): DataFrame com as arestas da rede.
    Returns:
        dict: Dicionário mapeando ID da comunidade para o nó principal.
    """
    # Cria o grafo direcionado
    grafo = nx.DiGraph()
    arestas = dados[['origem', 'destino']].values
    grafo.add_edges_from(arestas)

    # Converte para grafo não direcionado para detecção de comunidades
    grafo_nao_direcionado = grafo.to_undirected()

    # Usa louvain_partitions para detectar comunidades
    particoes = louvain_partitions(grafo_nao_direcionado, seed=42)  # seed para reprodutibilidade

    # Seleciona a primeira partição (ou a com maior modularidade, se necessário)
    particao = next(particoes)  # Pega a primeira partição

    # Organiza nós por comunidade
    comunidades = {}
    for id_comunidade, comunidade in enumerate(particao):
        comunidades[id_comunidade] = list(comunidade)

    # Encontra o nó principal por maior grau de entrada em cada comunidade
    nos_principais = {}
    for id_comunidade, nos in comunidades.items():
        graus = grafo.in_degree()
        nome_criterio = "maior grau de entrada"
        graus_comunidade = [(no, graus[no]) for no in nos if no in grafo.nodes()]
        if graus_comunidade:  # Verifica se a comunidade não está vazia
            no_principal = max(graus_comunidade, key=lambda x: x[1])[0]
            nos_principais[id_comunidade] = no_principal
            print(f"Comunidade {id_comunidade}: Nó {no_principal} com {nome_criterio}")
        else:
            print(f"Comunidade {id_comunidade}: Nenhuma aresta de entrada encontrada")

    print(f"Nós selecionados por {nome_criterio} são centrais em suas comunidades.")
    return nos_principais

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def exercicio_6(dados, nos_principais):
    """
    Plota o número de arestas de entrada ao longo de 803 dias para os nós principais.
    Args:
        dados (pd.DataFrame): DataFrame com as arestas da rede.
        nos_principais (dict): Nós com maior grau de entrada por comunidade.
    """
    dados['dia'] = dados['timestamp'] // (24 * 3600)

    plt.figure(figsize=(12, 6))
    for id_comunidade, no in nos_principais.items():
        arestas_no = dados[dados['destino'] == no]
        contagem_diaria = arestas_no.groupby('dia').size()
        contagem_diaria = contagem_diaria.reindex(range(803), fill_value=0)
        plt.plot(contagem_diaria.index, contagem_diaria.values, label=f"Nó {no} (Comunidade {id_comunidade})")

    plt.xlabel("Dia")
    plt.ylabel("Número de E-mails Recebidos")
    plt.title("E-mails Recebidos ao Longo do Tempo")
    plt.legend()
    nome_arquivo = os.path.join("figuras", "exercicio_06_temporal.png")
    plt.savefig(nome_arquivo)
    plt.close()
    print(f"Visualização temporal salva como '{nome_arquivo}'.")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
import os
import random

def exercicio_7(dados, nos_principais):
    """
    Decompõe séries temporais de dois nós escolhidos aleatoriamente, compara tendência e sazonalidade,
    e retorna os nós para uso posterior.
    Args:
        dados (pd.DataFrame): DataFrame com as arestas da rede (origem, destino, timestamp).
        nos_principais (dict): Nós com maior grau de entrada por comunidade.
    Returns:
        tuple: (no_a, no_b), IDs dos nós selecionados.
    """
    # Seleciona dois nós aleatoriamente
    random.seed(42)  # Para reprodutibilidade
    try:
        nos = random.sample(list(nos_principais.values()), k=2)
    except ValueError as e:
        print(f"Erro ao selecionar nós: {str(e)}. Usando nós padrão ou repetidos.")
        nos = list(nos_principais.values())[:2]
        if len(nos) < 2:
            nos = [nos[0], nos[0]] if nos else [0, 0]

    no_a, no_b = nos
    print(f"Nós selecionados: A={no_a}, B={no_b}")

    # Converte timestamps para dias
    dados['dia'] = dados['timestamp'] // (24 * 3600)
    max_dias = 803  # Período total da rede (fixo para consistência)

    def obter_serie_temporal(no):
        """Obtém a série temporal de arestas de entrada para um nó."""
        arestas_no = dados[dados['destino'] == no]
        contagem_diaria = arestas_no.groupby('dia').size()
        return contagem_diaria.reindex(range(max_dias), fill_value=0)

    try:
        # Obtém séries temporais
        serie_a = obter_serie_temporal(no_a)
        serie_b = obter_serie_temporal(no_b)

        # Decomposição: modelo aditivo, período semanal
        modelo = "additive"
        periodo_sazonal = 7
        decomp_a = seasonal_decompose(serie_a, model=modelo, period=periodo_sazonal, extrapolate_trend='freq')
        decomp_b = seasonal_decompose(serie_b, model=modelo, period=periodo_sazonal, extrapolate_trend='freq')

        # Gera visualizações
        for no, decomp in [(no_a, decomp_a), (no_b, decomp_b)]:
            plt.figure(figsize=(12, 8))
            plt.subplot(411)
            plt.plot(decomp.observed, label='Observado')
            plt.legend(loc='best')
            plt.subplot(412)
            plt.plot(decomp.trend, label='Tendência')
            plt.legend(loc='best')
            plt.subplot(413)
            plt.plot(decomp.seasonal, label='Sazonalidade')
            plt.legend(loc='best')
            plt.subplot(414)
            plt.plot(decomp.resid, label='Ruído')
            plt.legend(loc='best')
            nome_arquivo = os.path.join("figuras", f"exercicio_07_no_{no}.png")
            plt.suptitle(f"Decomposição da Série Temporal - Nó {no} (Período Semanal, Modelo Aditivo)")
            plt.tight_layout(rect=[0, 0, 1, 0.95])
            plt.savefig(nome_arquivo)
            plt.close()
            print(f"Gráfico salvo como '{nome_arquivo}'.")

        # Compara tendências e sazonalidades
        correlacao_tendencia = np.corrcoef(decomp_a.trend, decomp_b.trend)[0, 1]
        correlacao_sazonalidade = np.corrcoef(decomp_a.seasonal, decomp_b.seasonal)[0, 1]
        print(f"\nCorrelação entre nós A={no_a} e B={no_b}:")
        print(f"  Tendência: {correlacao_tendencia:.4f}")
        print(f"  Sazonalidade: {correlacao_sazonalidade:.4f}")

        # Conclusões interpretativas
        print("\nInterpretação no contexto da instituição:")
        if abs(correlacao_tendencia) > 0.7:
            print(f"A alta correlação de tendência ({correlacao_tendencia:.4f}) indica que A e B têm padrões de longo prazo semelhantes. Eles podem ser líderes ou departamentos centrais com fluxos de e-mails sincronizados, como grupos de pesquisa colaborativos.")
        else:
            print(f"A baixa correlação de tendência ({correlacao_tendencia:.4f}) sugere que A e B têm dinâmicas distintas. Eles podem representar áreas diferentes, como administração e pesquisa, com demandas de comunicação independentes.")

        if abs(correlacao_sazonalidade) > 0.7:
            print(f"A alta correlação de sazonalidade ({correlacao_sazonalidade:.4f}) mostra que A e B seguem ciclos semanais similares, possivelmente devido a rotinas institucionais compartilhadas, como reuniões ou relatórios.")
        else:
            print(f"A baixa correlação de sazonalidade ({correlacao_sazonalidade:.4f}) indica ciclos distintos, talvez com um nó ativo em dias úteis e outro com picos esporádicos, refletindo funções variadas.")

        print("Esses padrões sugerem estratégias para otimizar a comunicação, como sincronizar fluxos para nós correlacionados ou diversificar canais para nós independentes.")

    except ValueError as e:
        print(f"Erro na análise temporal: {str(e)}. Não foi possível completar a decomposição.")
        return None, None

    return no_a, no_b

In [ ]:
import pandas as pd
import random

def exercicio_8(dados, no_a, no_b):
    """
    Cria nó C e redireciona aleatoriamente 25% das arestas de A e B para C.
    Args:
        dados (pd.DataFrame): DataFrame com as arestas da rede.
        no_a (int): Nó A selecionado no Exercício 7.
        no_b (int): Nó B selecionado no Exercício 7.
    Returns:
        tuple: DataFrame modificado e ID do novo nó C.
    """
    print(f"Usando nós do Exercício 7: A={no_a}, B={no_b}")

    # Cria o novo nó C
    novo_no = max(dados['origem'].max(), dados['destino'].max()) + 1
    print(f"Criando novo nó C: {novo_no}")

    # Cria uma cópia do DataFrame para modificações
    dados_modificados = dados.copy()

    def redirecionar_arestas(no):
        """Redireciona aleatoriamente 25% das arestas destinadas ao nó para novo_no."""
        arestas_no = dados_modificados[dados_modificados['destino'] == no]
        num_arestas = len(arestas_no)
        num_redirecionar = int(0.25 * num_arestas)
        if num_redirecionar == 0:
            print(f"Nenhuma aresta redirecionada para nó {no} (menos de 4 arestas).")
            return

        # Seleciona aleatoriamente 25% das arestas
        indices = arestas_no.sample(n=num_redirecionar, random_state=42).index
        dados_modificados.loc[indices, 'destino'] = novo_no
        print(f"Redirecionadas {num_redirecionar} arestas do nó {no} por amostragem aleatória.")

    # Redireciona arestas para A e B
    redirecionar_arestas(no_a)
    redirecionar_arestas(no_b)

    return dados_modificados, novo_no

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
import os

def exercicio_9(dados_modificados, no_a, no_b, novo_no):
    """
    Repete a decomposição das séries temporais para os nós A, B e C após modificação, compara tendências e sazonalidades, e tira conclusões.
    Args:
        dados_modificados (pd.DataFrame): DataFrame com a rede modificada.
        no_a (int): Nó A do Exercício 7.
        no_b (int): Nó B do Exercício 7.
        novo_no (int): ID do novo nó C do Exercício 8.
    """
    nos = [no_a, no_b, novo_no]
    print(f"Nós analisados: A={no_a}, B={no_b}, C={novo_no}")

    # Converte timestamps para dias
    dados_modificados['dia'] = dados_modificados['timestamp'] // (24 * 3600)

    def obter_serie_temporal(no):
        """Obtém a série temporal de arestas de entrada para um nó."""
        arestas_no = dados_modificados[dados_modificados['destino'] == no]
        contagem_diaria = arestas_no.groupby('dia').size()
        return contagem_diaria.reindex(range(803), fill_value=0)

    try:
        # Configuração fixa: modelo aditivo, período 7 dias
        modelo = "additive"
        periodo = 7

        # Decompõe as séries temporais para A, B e C
        series = {no: obter_serie_temporal(no) for no in nos}
        decomps = {
            no: seasonal_decompose(serie, model=modelo, period=periodo, extrapolate_trend='freq')
            for no, serie in series.items()
        }

        # Gera visualizações para cada nó
        for no in nos:
            decomp = decomps[no]
            plt.figure(figsize=(12, 8))
            plt.subplot(411)
            plt.plot(decomp.observed, label='Observado')
            plt.legend()
            plt.subplot(412)
            plt.plot(decomp.trend, label='Tendência')
            plt.legend()
            plt.subplot(413)
            plt.plot(decomp.seasonal, label='Sazonalidade')
            plt.legend()
            plt.subplot(414)
            plt.plot(decomp.resid, label='Ruído')
            plt.legend()
            nome_arquivo = os.path.join("figuras", f"exercicio_09_no_{no}_modificado.png")
            plt.suptitle(f"Decomposição da Série Temporal para Nó {no} (Modificado, Período=7, Modelo=aditivo)")
            plt.savefig(nome_arquivo)
            plt.close()
            print(f"Decomposição salva como '{nome_arquivo}'.")

        # Compara tendências e sazonalidades (A vs. B, A vs. C, B vs. C)
        pares = [("A vs. B", no_a, no_b), ("A vs. C", no_a, novo_no), ("B vs. C", no_b, novo_no)]
        for nome_par, n1, n2 in pares:
            correlacao_tendencia = np.corrcoef(decomps[n1].trend, decomps[n2].trend)[0, 1]
            correlacao_sazonalidade = np.corrcoef(decomps[n1].seasonal, decomps[n2].seasonal)[0, 1]
            print(f"\nCorrelações para {nome_par}:")
            print(f"  Tendência: {correlacao_tendencia:.4f}")
            print(f"  Sazonalidade: {correlacao_sazonalidade:.4f}")

        # Conclusões interpretativas
        print("\nInterpretação dos resultados após redirecionamento:")
        # A vs. B
        correlacao_tendencia_ab = np.corrcoef(decomps[no_a].trend, decomps[no_b].trend)[0, 1]
        correlacao_sazonalidade_ab = np.corrcoef(decomps[no_a].seasonal, decomps[no_b].seasonal)[0, 1]
        if abs(correlacao_tendencia_ab) > 0.7:
            print(f"A alta correlação da tendência entre A e B ({correlacao_tendencia_ab:.4f}) sugere que o redirecionamento para C não alterou significativamente seus padrões de longo prazo. Eles continuam desempenhando papéis complementares.")
        else:
            print(f"A baixa correlação da tendência entre A e B ({correlacao_tendencia_ab:.4f}) indica que o redirecionamento pode ter diferenciado ainda mais suas dinâmicas de longo prazo, talvez reduzindo interdependências.")

        if abs(correlacao_sazonalidade_ab) > 0.7:
            print(f"A alta correlação da sazonalidade entre A e B ({correlacao_sazonalidade_ab:.4f}) implica que os ciclos semanais permanecem sincronizados, apesar do redirecionamento para C.")
        else:
            print(f"A baixa correlação da sazonalidade entre A e B ({correlacao_sazonalidade_ab:.4f}) sugere que o redirecionamento alterou os ciclos temporais, possivelmente redistribuindo picos de e-mails.")

        # Impacto em C
        correlacao_tendencia_ac = np.corrcoef(decomps[no_a].trend, decomps[novo_no].trend)[0, 1]
        correlacao_tendencia_bc = np.corrcoef(decomps[no_b].trend, decomps[novo_no].trend)[0, 1]
        correlacao_sazonalidade_ac = np.corrcoef(decomps[no_a].seasonal, decomps[novo_no].seasonal)[0, 1]
        correlacao_sazonalidade_bc = np.corrcoef(decomps[no_b].seasonal, decomps[novo_no].seasonal)[0, 1]

        if abs(correlacao_tendencia_ac) > 0.7 or abs(correlacao_tendencia_bc) > 0.7:
            print(f"A tendência de C é semelhante à de A ({correlacao_tendencia_ac:.4f}) ou B ({correlacao_tendencia_bc:.4f}), indicando que C assumiu parte do papel de longo prazo de um dos nós originais.")
        else:
            print(f"A tendência de C é distinta de A ({correlacao_tendencia_ac:.4f}) e B ({correlacao_tendencia_bc:.4f}), sugerindo que C opera de forma independente em longo prazo.")

        if abs(correlacao_sazonalidade_ac) > 0.7 or abs(correlacao_sazonalidade_bc) > 0.7:
            print(f"A sazonalidade de C é semelhante à de A ({correlacao_sazonalidade_ac:.4f}) ou B ({correlacao_sazonalidade_bc:.4f}), mostrando que C herdou ciclos semanais de pelo menos um dos nós.")
        else:
            print(f"A sazonalidade de C é distinta de A ({correlacao_sazonalidade_ac:.4f}) e B ({correlacao_sazonalidade_bc:.4f}), indicando que C tem padrões cíclicos próprios.")

        print("O redirecionamento para C pode ter redistribuído a carga de comunicação, afetando estratégias institucionais, como balanceamento de servidores ou fluxos de e-mails.")

    except ValueError as e:
        print(f"Erro na decomposição: {str(e)}. Não foi possível completar a análise.")

In [ ]:
import pandas as pd
from scipy import stats

def exercicio_10(dados, dados_modificados, no_a, no_b, novo_no, teste="t_test"):
    """
    Analisa mudanças no fluxo de e-mails para A, B e C com testes estatísticos.
    Args:
        dados (pd.DataFrame): DataFrame original.
        dados_modificados (pd.DataFrame): DataFrame com a rede modificada.
        no_a (int): Nó A do Exercício 7.
        no_b (int): Nó B do Exercício 7.
        novo_no (int): ID do novo nó C do Exercício 8.
        teste (str): 't_test' ou 'mann_whitney'.
    """
    print(f"Analisando nós: A={no_a}, B={no_b}, C={novo_no}")

    # Calcula grau de entrada total
    grau_entrada_antes = dados.groupby('destino').size()
    grau_entrada_depois = dados_modificados.groupby('destino').size()

    def obter_grau_entrada(no):
        return grau_entrada_antes.get(no, 0), grau_entrada_depois.get(no, 0)

    grau_a_antes, grau_a_depois = obter_grau_entrada(no_a)
    grau_b_antes, grau_b_depois = obter_grau_entrada(no_b)
    grau_c_antes, grau_c_depois = obter_grau_entrada(novo_no)

    print(f"Nó A grau de entrada: Antes={grau_a_antes}, Depois={grau_a_depois}")
    print(f"Nó B grau de entrada: Antes={grau_b_antes}, Depois={grau_b_depois}")
    print(f"Nó C grau de entrada: Antes={grau_c_antes}, Depois={grau_c_depois}")

    # Verifica redução em A e B, aumento em C
    reduziu_a = grau_a_depois < grau_a_antes
    reduziu_b = grau_b_depois < grau_b_antes
    aumentou_c = grau_c_depois > grau_c_antes
    if reduziu_a and reduziu_b:
        print("O fluxo de e-mails para A e B diminuiu após introdução do nó C.")
        if aumentou_c:
            print("O nó C absorveu parte do fluxo, como esperado.")
        else:
            print("O nó C não registrou aumento correspondente, sugerindo possíveis discrepâncias.")
    else:
        print("O fluxo de e-mails para A e/ou B não diminuiu consistentemente.")

    # Séries temporais diárias
    dados['dia'] = dados['timestamp'] // (24 * 3600)
    dados_modificados['dia'] = dados_modificados['timestamp'] // (24 * 3600)

    def obter_grau_entrada_diario(no, df):
        arestas_no = df[df['destino'] == no]
        return arestas_no.groupby('dia').size().reindex(range(803), fill_value=0)

    # Gera séries para A, B e C
    nos = [(no_a, "A"), (no_b, "B"), (novo_no, "C")]
    series = {
        nome: {
            "antes": obter_grau_entrada_diario(no, dados),
            "depois": obter_grau_entrada_diario(no, dados_modificados)
        }
        for no, nome in nos
    }

    # Testes de hipóteses
    alfa = 0.05
    for no, nome in nos:
        serie_antes = series[nome]["antes"]
        serie_depois = series[nome]["depois"]
        try:
            if teste == "t_test":
                estatistica, valor_p = stats.ttest_rel(serie_antes, serie_depois)
                nome_teste = "teste t pareado"
            elif teste == "mann_whitney":
                estatistica, valor_p = stats.mannwhitneyu(serie_antes, serie_depois, alternative='two-sided')
                nome_teste = "teste de Mann-Whitney U"
            else:
                print("Teste inválido. Usando t_test.")
                estatistica, valor_p = stats.ttest_rel(serie_antes, serie_depois)
                nome_teste = "teste t pareado"

            print(f"Nó {nome} ({no}) {nome_teste}: estatística={estatistica:.4f}, p-valor={valor_p:.4f}")
            if valor_p < alfa:
                print(f"Mudança significativa no fluxo de e-mails do nó {nome} (p < {alfa}).")
            else:
                print(f"Sem mudança significativa no fluxo de e-mails do nó {nome} (p >= {alfa}).")

        except ValueError as e:
            print(f"Erro no teste para nó {nome}: {str(e)}. Pulando análise estatística.")

    # Conclusões institucionais
    print("\nConclusões para a instituição:")
    if reduziu_a and reduziu_b and aumentou_c:
        print("O redirecionamento para o nó C foi eficaz em reduzir a carga de e-mails em A e B, transferindo parte do fluxo para C. Isso pode aliviar servidores ou administradores sobrecarregados, melhorando a eficiência da comunicação.")
        if series["A"]["depois"].mean() < series["A"]["antes"].mean() and series["B"]["depois"].mean() < series["B"]["antes"].mean():
            print("Testes confirmam redução significativa no fluxo diário, sugerindo que C assumiu responsabilidades de comunicação.")
        else:
            print("Embora o fluxo total tenha diminuído, os padrões diários não mudaram significativamente, indicando que a redução pode ser distribuída irregularmente.")
    else:
        print("O redirecionamento não reduziu consistentemente o fluxo para A e B, ou C não absorveu o fluxo esperado. Isso pode indicar que a intervenção não foi suficiente para balancear a comunicação.")
        print("Recomenda-se revisar a proporção de redirecionamento (25%) ou considerar outros nós para redistribuição.")

    print("Esses resultados podem orientar ajustes na infraestrutura de e-mails, como adicionar mais nós ou otimizar fluxos para evitar gargalos.")

## 🚀 Execução Automática dos Exercícios

In [ ]:
# Cria a pasta 'figuras' se não existir
os.makedirs("figuras", exist_ok=True)

# Exercício 1 - Carregamento
print("Exercício 1: Carregando dados...")
dados = carregar_dados("email-Eu-core-temporal.txt.gz")

# Exercício 2 - Visualização
print("\nExercício 2: Visualizando rede...")
exercicio_2(dados)

# Exercício 3 - Análise global
print("\nExercício 3: Analisando média dos menores caminhos...")
exercicio_3(dados)

# Exercício 4 - Análise estrutural
print("\nExercício 4: Calculando centralidade de intermediação...")
exercicio_4(dados)

# Exercício 5 - Comunidades
print("\nExercício 5: Detectando comunidades...")
nos_principais = exercicio_5(dados)

# Exercício 6 - Análise temporal
print("\nExercício 6: Visualizando comportamento temporal...")
exercicio_6(dados, nos_principais)

# Exercício 7 - Séries temporais
print("\nExercício 7: Decompondo séries temporais...")
no_a, no_b = exercicio_7(dados, nos_principais)

# Exercício 8 - Modificação da rede
print("\nExercício 8: Modificando a rede...")
dados_modificados, novo_no = exercicio_8(dados, no_a, no_b)

# Exercício 9 - Análise da rede modificada
print("\nExercício 9: Decompondo séries após modificação...")
exercicio_9(dados_modificados, no_a, no_b, novo_no)

# Exercício 10 - Teste de hipóteses
print("\nExercício 10: Analisando impacto estatístico da mudança...")
exercicio_10(dados, dados_modificados, no_a, no_b, novo_no, teste="t_test")

Exercício 1: Carregando dados...
Dados convertidos e salvos como: email-Eu-core-temporal.csv
Dataset carregado com 332334 arestas.

Exercício 2: Visualizando rede...
Visualização da rede salva como 'figuras/exercicio_02_rede_spring.png'.

Exercício 3: Analisando média dos menores caminhos...
Média dos menores caminhos (maior componente fortemente conexo): 2.5475

Análise da conectividade e eficiência:
A média de 2.5475 indica alta conectividade (característica de mundo pequeno).
E-mails tendem a alcançar destinatários com poucos intermediários, sugerindo comunicação eficiente.
Em redes sociais, médias abaixo de 6 são comuns, como no conceito de 'seis graus de separação'.

Exercício 4: Calculando centralidade de intermediação...
Top 5 nós por centralidade de intermediação:
Nó 90: 0.0749
Nós com alta intermediação conectam grupos distintos.
Nó 951: 0.0389
Nós com alta intermediação conectam grupos distintos.
Nó 2: 0.0280
Nós com alta intermediação conectam grupos distintos.
Nó 120: 0.025